In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/t5-small-finetuned-spider.zip'

Mounted at /content/drive


In [3]:
!unzip -q "/content/drive/MyDrive/t5-small-finetuned-spider.zip" -d "/content/t5-small-finetuned-spider"

Archive:  /content/drive/MyDrive/t5-small-finetuned-spider.zip
  inflating: t5-small-finetuned-spider/added_tokens.json  
  inflating: t5-small-finetuned-spider/config.json  
  inflating: t5-small-finetuned-spider/generation_config.json  
  inflating: t5-small-finetuned-spider/model.safetensors  
  inflating: t5-small-finetuned-spider/special_tokens_map.json  
  inflating: t5-small-finetuned-spider/spiece.model  
  inflating: t5-small-finetuned-spider/tokenizer_config.json  


In [5]:
CKPT = f'/content/t5-small-finetuned-spider'
from transformers import AutoTokenizer, T5ForConditionalGeneration
tokenizer = AutoTokenizer.from_pretrained(CKPT)
model = T5ForConditionalGeneration.from_pretrained(CKPT)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 909.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 944.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 1.1 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
test_data = load_dataset('wikisql', split='test')

def translate_to_sql(text):
    inputs = tokenizer(text, padding='longest', max_length=64, return_tensors='pt')
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask
    output = model.generate(input_ids, attention_mask=attention_mask, max_length=64)

    return tokenizer.decode(output[0], skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/datasets/load.py:1461: FutureWarning: The repository for wikisql contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikisql
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


Generating test split:   0%|          | 0/15878 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8421 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/56355 [00:00<?, ? examples/s]

In [55]:
expected_sqls = [test_data[i]['sql']['human_readable'] for i in range((50))]
predicted_sqls = [translate_to_sql('translate to SQL: ' + test_data[i]['question']) for i in range(50)]

In [49]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [57]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [56]:
predicted_sqls_lower = [sql.lower() for sql in predicted_sqls]
expected_sqls_lower = [sql.lower() for sql in expected_sqls]

In [59]:
from nltk.tokenize import word_tokenize

def normalize_sql(sql):
    # Remove special characters and lowercase the SQL query
    return ''.join(e for e in sql if e.isalnum() or e.isspace()).lower()

def tokenize_sql(sql):
    # Tokenize the normalized SQL query
    return word_tokenize(sql)

def calculate_accuracy(predicted_sql, expected_sql):
    # Normalize and tokenize the SQL queries
    predicted_tokens = tokenize_sql(normalize_sql(predicted_sql))
    expected_tokens = tokenize_sql(normalize_sql(expected_sql))

    # Calculate the number of common tokens
    common_tokens = len(set(predicted_tokens) & set(expected_tokens))

    # Calculate the accuracy as the ratio of common tokens to total tokens
    accuracy = common_tokens / len(expected_tokens)

    return accuracy

def calculate_average_accuracy(predicted_sqls_lower, expected_sqls_lower):
    total_accuracy = 0
    for predicted_sql, expected_sql in zip(predicted_sqls_lower, expected_sqls_lower):
        total_accuracy += calculate_accuracy(predicted_sql, expected_sql)
    return total_accuracy / len(predicted_sqls_lower)

def calculate_precision_recall_f1(predicted_sql, expected_sql):
    predicted_tokens = tokenize_sql(normalize_sql(predicted_sql))
    expected_tokens = tokenize_sql(normalize_sql(expected_sql))

    # Calculate precision
    if len(predicted_tokens) == 0:
        precision = 0
    else:
        precision = len(set(predicted_tokens) & set(expected_tokens)) / len(predicted_tokens)

    # Calculate recall
    if len(expected_tokens) == 0:
        recall = 0
    else:
        recall = len(set(predicted_tokens) & set(expected_tokens)) / len(expected_tokens)

    # Calculate F1 score
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    return precision, recall, f1_score

def calculate_average_precision_recall_f1(predicted_sqls_lower, expected_sqls_lower):
    total_precision = 0
    total_recall = 0
    total_f1_score = 0
    for predicted_sql, expected_sql in zip(predicted_sqls_lower, expected_sqls_lower):
        precision, recall, f1_score = calculate_precision_recall_f1(predicted_sql, expected_sql)
        total_precision += precision
        total_recall += recall
        total_f1_score += f1_score
    num_queries = len(predicted_sqls_lower)
    average_precision = total_precision / num_queries
    average_recall = total_recall / num_queries
    average_f1_score = total_f1_score / num_queries
    return average_precision, average_recall, average_f1_score

average_accuracy = calculate_average_accuracy(predicted_sqls_lower, expected_sqls_lower)
average_precision, average_recall, average_f1_score = calculate_average_precision_recall_f1(predicted_sqls_lower, expected_sqls_lower)
print("Average Accuracy:", average_accuracy)
print("Average Precision:", average_precision)
print("Average Recall:", average_recall)
print("Average F1 Score:", average_f1_score)

Average Accuracy: 0.6081106181400299
Average Precision: 0.6650121267621267
Average Recall: 0.6081106181400299
Average F1 Score: 0.6181417628036352
